In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.io import ascii

In [ ]:
#Paths
base_folder = '/home/stba7609/'
secreto_path = '/home/stba7609/SECRETO/'

In [ ]:
#Reading the models
obs_spectrum_he3p3_200_path = base_folder + "NII_nebular_phase/SUMO_results/200d/spectrum_he3p3_AJ_40_60.datrun001"
thin_spectrum_he3p3_200_path = "tc.dat"

obs_spectrum_he8p0_path = base_folder + "NII_nebular_phase/SUMO_results/350d/spectrum_he8p0_AJ_10_90.datrun001"

obs_33_200 = ascii.read(obs_spectrum_he3p3_200_path)
thin_33_200 = ascii.read(thin_spectrum_he3p3_200_path)
obs_80 = ascii.read(obs_spectrum_he8p0_path)

In [ ]:
#Observation names
epochs_nii = np.array([157, 179, 196, 241, 271, 
                       288, 295, 348, 346])

sn_names_with_nii = np.array(['SN2008bo', 'SN2013df', 'SN1996cb', 'SN1997dq', 'SNMASTEROTJ1204',
                             'SN2007Y', 'SN1985F', 'SN2011hs', 'iPTF13bvn'])


#Create arrays with file names for the different types
Ib_names = np.array(['SN1985F', 'SN1996aq', 'SN2004ao', 'SN2009jf',
                    'SN2015ah', 'SN2004dk', 'SN2012au', 'SNMASTEROTJ1204'
                     , 'SN2008D', 'SN2007Y', 'SN2007C', 'SN2006ld', 'SN2004gq',
                    'SN2019odp'])
Ic_names = np.array(['SN1987M', 'SN1997dq', 'SN1998bw', 'SN2003gf', 'SN2004aw', 'SN2007gr', 'SN2007I',
                    'SN2011bm', 'SN2013ge', 'SN2015fn', 'SN2019yz', 'SNPTF12gzk'])
IIb_names = np.array(['SN1993J', 'SN1996cb', 'SN2001ig', 'SN2003bg', 'SN2008ax',
                      'SN2008aq', 'SN2008bo', 'SN2009K', 'SN2011dh', 'SN2011ei', 'SN2011hs',
                      'SN2013df', 'SN2020acat','SN2022crv', 'SNASASSN14az', 'SNPTF12os', 'iPTF13bvn'])

In [ ]:
#The part plotting the models


zoom = [5200, 6200]

fig, ax = plt.subplots(1, 2)

plt.subplots_adjust(wspace=0, hspace=0)

filter1 = (obs_33_200['col1'] < 5950)*(obs_33_200['col1'] > 5850)

norm = np.max(obs_33_200['col2'][filter1])

ax[1].plot(obs_33_200['col1'], obs_33_200['col2']/norm + 1, c = 'k', label = 'Total Emergent flux')
ax[1].plot(obs_33_200['col1'], obs_33_200['col15']/norm + 1, c = 'red', label = 'Emergent [N II] flux')
ax[1].plot(thin_33_200['col1'], thin_33_200['col12']/norm + 1, c = 'gray', label = 'Emitted [N II] flux ')
ax[1].plot(obs_33_200['col1'], obs_33_200['col59']/norm + 1, c = 'blue', label = 'Emergent Fe II flux')

filter2 = (obs_80['col1'] < 5950)*(obs_80['col1'] > 5850)
norm2 = np.max(obs_80['col2'][filter2])

ax[1].plot(obs_80['col1'], obs_80['col2']/norm2  , c = 'k')
ax[1].plot(obs_80['col1'], obs_80['col15']/norm2 , c = 'red')
ax[1].plot(obs_80['col1'], obs_80['col59']/norm2 , c = 'blue')



ax[1].axvline(x = 5600, linestyle = '--', alpha = 0.5, c = 'r')
ax[1].axvline(x = 5754, linestyle = '--', alpha = 0.5, c = 'gray')
ax[1].axhline(y = 1, c = 'black', lw = 3)


ax[1].set_xlim(zoom[0], zoom[1])
ax[1].set_ylim(0, 2)

plt.figlegend(bbox_to_anchor=(0.23, 0.92, 1, 0), loc= 'center',
              borderaxespad = 0, handletextpad = 0.3, ncol=2 ,columnspacing=1.5, frameon= False)

fig.set_size_inches(8, 6)


#===========================================================================================================
offset = 4.3

for i in range(len(sn_names_with_nii)):
    
    #Read in the specific SN folder
    sn_name = sn_names_with_nii[i]
    if sn_name in Ib_names:
        obs_path = secreto_path + 'WISEREP_Ib/Standardised_spectra/'
    elif sn_name in Ic_names:
        obs_path = secreto_path + 'WISEREP_Ic/Standardised_spectra/'
    elif sn_name in IIb_names:
        obs_path = secreto_path + 'WISEREP_IIb/Standardised_spectra/'
        
    observations = os.listdir(obs_path)
    
    this_sn_fnames = []
    for j in range(len(observations)):
        if sn_name in observations[j]:
            this_sn_fnames.append(observations[j])
            
    this_sn_fnames = sorted(this_sn_fnames)
    local_epochs = [] #days, time since explosion
    local_NII_fluxes, local_NII_sigs = [], []
    
    wl_list, flux_list = [], []
    
    for k in range(len(this_sn_fnames)):
        
        this_sn_fname = this_sn_fnames[k]
        
        if 'discovery' in this_sn_fname:
            local_epochs.append(int(this_sn_fname[-7:-4])) #The epoch is always at this position in the fname
        else: #For peaklight, add 20 days to get to explosion epoch
            local_epochs.append(int(this_sn_fname[-7:-4]) + 20)
            
        
        if local_epochs[k] in epochs_nii:
            
            #Read in the data
            this_observation_data = np.loadtxt(obs_path + this_sn_fname, delimiter = ',')
            wl, flux = this_observation_data[:, 0], this_observation_data[:, 1]
            
            norm_filter = (wl < 5950) * (wl > 5850)

            if 'SNMASTEROTJ1204' in this_sn_fname:
                ax[0].plot(wl, flux/np.max(flux[norm_filter])+ offset, label = 'J1204' + '_' + str(epochs_nii[i]) + 'd')
            elif 'iPTF' in this_sn_fname:
                ax[0].plot(wl, flux/np.max(flux[norm_filter])+ offset, label = '13bvn' + '_' + str(epochs_nii[i]) + 'd')
            else:
                ax[0].plot(wl, flux/np.max(flux[norm_filter])+ offset, label = sn_name[2:] + '_' + str(epochs_nii[i]) + 'd')
            
            
    if i == 1 or i == 4:
        offset -= 0.8
    else:
        offset -= 0.5
    
                
ax[0].axvline(x = 5754, linestyle = '--', c = 'grey', alpha = 0.5)
ax[0].axvline(x = 5600, linestyle = '--', c = 'r', alpha = 0.5)
ax[0].set_xlim(5200, 6200)
ax[0].set_ylim(0, 5.4)

ax[0].legend(bbox_to_anchor=(0.0, 1.12, 1, 0), loc= 'center',
              borderaxespad = 0, handletextpad = 0.3, ncol=2 ,columnspacing=1.5, frameon= False)


ax[1].set_xticklabels(['', '5400', '5600', '5800', '6000', '6200'])
ax[0].set_yticks([])
ax[1].set_yticks([])

fig.text(0.08, 0.36, s = 'Flux + constant [a.u.]', rotation = 90, fontsize = 14)
fig.text(0.4, 0.04, s = 'Rest Wavelength [Å]', fontsize = 14)
fig.text(0.27, 0.385, s = '[N II] 5754', c = 'r')
fig.text(0.38, 0.42, s = 'Na I D', c = 'r')
fig.text(0.515, 0.51, s = 'he3p3_200d', c = 'r', fontsize = 16)
fig.text(0.515, 0.45, s = 'he8p0_350d', c = 'r', fontsize = 16)

plt.savefig('Figure_11.pdf',  bbox_inches = 'tight')
plt.show()